In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import friedmanchisquare, wilcoxon
from mapper import continent_mapper, fullname_to_short, politicians_db

import warnings
warnings.filterwarnings('ignore')

# Set style for plots
plt.style.use('default')
sns.set_palette("husl")

In [12]:
def is_nan_or_list_with_nan(obj):
    try:
        if any(pd.isna(elem) for elem in obj):
            return True
    except TypeError:
        if pd.isna(obj):
            return True
    return False

In [ ]:
class ComprehensivePoliticalAnalyzer:
    def __init__(self, phase2_files):
        """
        Initialize the analyzer with both phase datasets
        """
        self.continent_mapping = continent_mapper
        self.politician_info = politicians_db
        self.phase2_df = self.load_and_clean_data(phase2_files, phase=2)
    
    def load_and_clean_data(self, filepath, phase):
        """
        Load and clean the data, with politician identification
        """
        dfs = []  # list to collect all DataFrames

        for path in filepath:
            df = pd.read_csv(path)

            # Extract politician short name from file path, e.g., "phase1_data/albanese_phase1_data.csv"
            politician_name = path.split("/")[1].split('_')[0].lower()
            df['politician_id'] = politician_name

            if phase == 2:
                # Clean phase 2 ranking columns
                ranking_cols = [
                    'competence', 'likable', 'trustworthy', 'approachable', 
                    'charismatic', 'aggressive', 'friendly', 'professional', 
                    'support', 'positive', 'visually_appealing', 'overall_attention'
                ]
                for col in ranking_cols:
                    if col in df.columns:
                        df[col] = df[col].apply(self.parse_ranking)

            # Add continent based on country
            df['continent'] = df['country'].map(self.continent_mapping)
            df['phase'] = phase

            # Add politician attributes
            for idx, row in df.iterrows():
                pid = row.get('politician_id', 'unknown')
                if pid in self.politician_info:
                    pdata = self.politician_info[pid]
                    df.loc[idx, 'politician_name'] = pdata['name']
                    df.loc[idx, 'politician_country'] = pdata['country']
                    df.loc[idx, 'politician_continent'] = pdata['continent']
                    df.loc[idx, 'ideology'] = pdata['ideology']
                    df.loc[idx, 'popularity_score'] = pdata['popularity_score']
                    df.loc[idx, 'incumbent_status'] = pdata['incumbent_status']
                    df.loc[idx, 'same_country'] = row['country'] == pdata['country']
                    df.loc[idx, 'same_continent'] = df.loc[idx, 'continent'] == pdata['continent']

            dfs.append(df)

        # Combine all individual DataFrames into one
        combined_df = pd.concat(dfs, ignore_index=True)
        return combined_df
    
    def parse_ranking(self, ranking_str):
        """Parse ranking string and return list of ranks"""
        if pd.isna(ranking_str) or ranking_str == '':
            return np.nan
        
        try:
            ranks = [int(x.strip()) for x in str(ranking_str).split(',') if x.strip()]
            return ranks
        except:
            return np.nan
    
    def extract_phase2_rankings_by_politician(self, trait_cols):
        """
        Extract rankings from Phase 2 data, organized by politician
        """
        results = []
        
        for idx, row in self.phase2_df.iterrows():
            for trait in trait_cols:
                if trait in self.phase2_df.columns and not is_nan_or_list_with_nan(row[trait]):
                    ranks = row[trait]
                    if isinstance(ranks, list) and len(ranks) == 4:
                        result_row = {
                            'response_id': row['response_id'],
                            'politician_id': row.get('politician_id', 'unknown'),
                            'politician_name': row.get('politician_name', 'Unknown'),
                            'age': row['age'],
                            'sex': row['sex'],
                            'participant_country': row['country'],
                            'participant_continent': row['continent'],
                            'politician_country': row.get('politician_country', 'Unknown'),
                            'politician_continent': row.get('politician_continent', 'Unknown'),
                            'same_country': row.get('same_country', False),
                            'same_continent': row.get('same_continent', False),
                            'ideology': row.get('ideology', 'unknown'),
                            'popularity_score': row.get('popularity_score', 0),
                            'political_engagement': row['political_engagement'],
                            'political_alignment': row['political_alignment'],
                            'familiarity': row['familiarity'],
                            'trait': trait,
                            'original_rank': ranks[0],
                            'background_change_rank': ranks[1],
                            'text_change_rank': ranks[2],
                            'font_change_rank': ranks[3]
                        }
                        results.append(result_row)
        
        return pd.DataFrame(results)
    
    def analyze_by_politician_strategy(self, strategy='pooled'):
        """
        Choose analysis strategy: 'pooled', 'individual', or 'comparative'
        """
        key_traits = ["competence", "likable", "trustworthy", "approachable", "charismatic",
                     "aggressive", "friendly", "professional", "support", "positive",
                     "visually_appealing", "overall_attention"]

        ranking_df = self.extract_phase2_rankings_by_politician(key_traits)
        print(f"Number of unique participants: {len(ranking_df['response_id'].unique())}")
        
        if ranking_df.empty:
            print("No valid ranking data found")
            return
        
        print(f"="*80)
        print(f"ANALYSIS STRATEGY: {strategy.upper()}")
        print(f"="*80)
        
        if strategy == 'pooled':
            self._analyze_pooled_data(ranking_df, key_traits)
        elif strategy == 'individual':
            self._analyze_individual_politicians(ranking_df, key_traits)
        elif strategy == 'comparative':
            self._analyze_comparative_politicians(ranking_df, key_traits)
        else:
            print(f"Unknown strategy: {strategy}")
    
    def _analyze_pooled_data(self, ranking_df, key_traits):
        """
        Analyze all politicians together (main effects)
        """
        print("POOLED ANALYSIS - All Politicians Combined")
        print("-" * 50)
        
        for trait in key_traits:
            trait_data = ranking_df[ranking_df['trait'] == trait]
            if trait_data.empty:
                continue
            
            print(f"\n{trait.upper()} Analysis:")
            
            # Main effects (all politicians pooled)
            conditions = ['original_rank', 'background_change_rank', 'text_change_rank', 'font_change_rank']
            condition_names = ['Original', 'Background Change', 'Text Change', 'Font Change']
            
            ranks_by_condition = {}
            for condition in conditions:
                ranks_by_condition[condition] = trait_data[condition].dropna()
            
            # Friedman test for overall differences
            valid_ranks = [ranks for ranks in ranks_by_condition.values() if len(ranks) > 0]
            if len(valid_ranks) >= 3:
                try:
                    stat, p_val = friedmanchisquare(*valid_ranks)
                    print(f"Friedman test: χ² = {stat:.3f}, p = {p_val:.3f}")
                    
                    if p_val < 0.05:
                        print("Significant overall differences detected")
                        # Post-hoc pairwise comparisons
                        self._pairwise_comparisons(trait_data, conditions)
                    else:
                        print("No significant overall differences")
                except Exception as e:
                    print(f"Error in Friedman test: {e}")
            
            # Show descriptive statistics
            print("\nDescriptive Statistics:")
            for condition, name in zip(conditions, condition_names):
                ranks = trait_data[condition].dropna()
                if len(ranks) > 0:
                    print(f"{name}: M = {ranks.mean():.2f}, SD = {ranks.std():.2f}, n = {len(ranks)}")
    
    def _analyze_individual_politicians(self, ranking_df, key_traits):
        """
        Analyze each politician separately
        """
        print("INDIVIDUAL POLITICIAN ANALYSIS")
        print("-" * 50)
        
        politicians = ranking_df['politician_name'].unique()
        
        for politician in politicians:
            if pd.isna(politician):
                continue
                
            print(f"\n{'='*60}")
            print(f"POLITICIAN: {politician}")
            print(f"{'='*60}")
            
            politician_data = ranking_df[ranking_df['politician_name'] == politician]
            
            # Get politician attributes
            sample_row = politician_data.iloc[0]
            popularity = sample_row.get('popularity_score', 'Unknown')
            ideology = sample_row.get('ideology', 'Unknown')
            pol_country = sample_row.get('politician_country', 'Unknown')
            
            print(f"Country: {pol_country}")
            print(f"Ideology: {ideology}")
            print(f"Popularity Score: {popularity}")
            print(f"Sample Size: {len(politician_data['response_id'].unique())} participants")
            
            for trait in key_traits:
                trait_data = politician_data[politician_data['trait'] == trait]
                if len(trait_data) < 5:  # Minimum sample size
                    continue
                
                print(f"\n--- {trait.upper()} ---")
                
                # Analyze each comparison
                comparisons = [
                    ('original_rank', 'background_change_rank', 'Background Color'),
                    ('original_rank', 'text_change_rank', 'Text Color'),
                    ('original_rank', 'font_change_rank', 'Font Style')
                ]
                
                for cond1, cond2, name in comparisons:
                    ranks1 = trait_data[cond1].dropna()
                    ranks2 = trait_data[cond2].dropna()
                    
                    if len(ranks1) >= 5 and len(ranks2) >= 5:
                        try:
                            stat, p_val = wilcoxon(ranks1, ranks2, alternative='two-sided')
                            effect_size = (ranks1.mean() - ranks2.mean()) / np.sqrt((ranks1.var() + ranks2.var()) / 2)
                            
                            print(f"{name}: Original M={ranks1.mean():.2f}, Changed M={ranks2.mean():.2f}")
                            print(f"  Wilcoxon: W={stat:.1f}, p={p_val:.3f}, d={effect_size:.3f}")
                            
                            if p_val < 0.05:
                                direction = "favors original" if ranks1.mean() < ranks2.mean() else "favors change"
                                print(f"  **Significant effect {direction}**")
                            else:
                                print(f"No significant effect")
                        except Exception as e:
                            print(f"{name}: Unable to compute statistics ({e})")
    
    def _analyze_comparative_politicians(self, ranking_df, key_traits):
        """Compare effects across politicians (background, text, font) for key traits."""
        print("COMPARATIVE POLITICIAN ANALYSIS")
        print("-" * 50)

        politicians = [p for p in ranking_df['politician_name'].unique() if not pd.isna(p)]
        if len(politicians) < 2:
            print("Need at least 2 politicians for comparative analysis")
            return

        print(f"Comparing {len(politicians)} politicians: {', '.join(politicians)}")

        # Calculate effect sizes for each politician
        politician_effects = {}
        manipulations = [
            ('bg_effect', 'background_change_rank', 'Background Color'),
            ('text_effect', 'text_change_rank', 'Text Color'),
            ('font_effect', 'font_change_rank', 'Font Style')
        ]
        
        focus_trait = ['trustworthy', 'likable', 'competence']

        for politician in politicians:
            pol_data = ranking_df[ranking_df['politician_name'] == politician]
            sample_row = pol_data.iloc[0]
            effects = {
                'popularity': sample_row.get('popularity_score', 0),
                'ideology': sample_row.get('ideology', 'unknown'),
                'country': sample_row.get('politician_country', 'Unknown')
            }
            for trait in key_traits:
                trait_data = pol_data[pol_data['trait'] == trait]
                original = trait_data['original_rank'].dropna()
                for effect_key, changed_col, _ in manipulations:
                    changed = trait_data[changed_col].dropna()
                    if len(original) >= 5 and len(changed) >= 5:
                        effects[f'{trait}_{effect_key}'] = original.mean() - changed.mean()
            politician_effects[politician] = effects

        # Print summary tables for each manipulation
        for effect_key, _, label in manipulations:
            print(f"\n{label} Effects by Politician:")
            print(f"{'Politician':<20} {'Popularity':<10} {'Ideology':<14} " +
                " ".join([f"{t.capitalize():<12}" for t in focus_trait]))
            print("-" * 80)
            for politician in politicians:
                effects = politician_effects[politician]
                popularity = effects.get('popularity', 0)
                ideology = str(effects.get('ideology', 'unknown'))[:14]
                trait_vals = [effects.get(f'{trait}_{effect_key}', 0) for trait in focus_trait]
                print(f"{fullname_to_short.get(politician, politician):<20} {popularity:<10} {ideology:<14} " +
                    " ".join([f"{v:>11.2f}" for v in trait_vals]))
                
        # Correlation with popularity
        self._analyze_popularity_correlations(politician_effects, key_traits)
    
    def _analyze_popularity_correlations(self, politician_effects, key_traits):
        """
        Analyze correlations between politician popularity and visual effects
        """
        print(f"\nPopularity-Effect Correlations:")
        print("-" * 40)
        
        for trait in key_traits:
            popularity_scores = []
            bg_effects = []
            
            for politician, effects in politician_effects.items():
                if f'{trait}_bg_effect' in effects and 'popularity' in effects:
                    popularity_scores.append(effects['popularity'])
                    bg_effects.append(effects[f'{trait}_bg_effect'])
            
            if len(popularity_scores) >= 3:
                correlation, p_val = stats.pearsonr(popularity_scores, bg_effects)
                print(f"{trait.capitalize()}: r = {correlation:.3f}, p = {p_val:.3f}")
                
                if abs(correlation) > 0.5 and p_val < 0.1:
                    direction = "more popular politicians benefit more" if correlation > 0 else "less popular politicians benefit more"
                    print(f"  Moderate correlation: {direction} from original design")
    
    def analyze_geographical_effects(self):
        """
        Analyze geographical effects (RQ4 enhanced)
        """
        print(f"\n{'='*80}")
        print("ENHANCED GEOGRAPHICAL ANALYSIS")
        print(f"{'='*80}")
        
        key_traits = ["competence", "likable", "trustworthy", "approachable", "charismatic",
                     "aggressive", "friendly", "professional", "support", "positive",
                     "visually_appealing", "overall_attention"]
        ranking_df = self.extract_phase2_rankings_by_politician(key_traits)
        
        if ranking_df.empty:
            return
        
        # Same country vs different country effects
        print("\nSAME CONTINENT vs DIFFERENT CONTINENT EFFECTS:")
        print("-" * 50)
        
        for trait in key_traits:
            trait_data = ranking_df[ranking_df['trait'] == trait]
            
            same_continent = trait_data[trait_data['same_continent'] == True]
            diff_continent = trait_data[trait_data['same_continent'] == False]
            
            if len(same_continent) >= 5 and len(diff_continent) >= 5:
                print(f"\n{trait.upper()}:")
                
                # Compare background effects
                same_bg_effect = same_continent['original_rank'].mean() - same_continent['background_change_rank'].mean()
                diff_bg_effect = diff_continent['original_rank'].mean() - diff_continent['background_change_rank'].mean()
                
                print(f"Same continent participants: Background effect = {same_bg_effect:.3f} (n={len(same_continent)})")
                print(f"Different continent participants: Background effect = {diff_bg_effect:.3f} (n={len(diff_continent)})")
                
                # Statistical test
                same_effects = same_continent['original_rank'] - same_continent['background_change_rank']
                diff_effects = diff_continent['original_rank'] - diff_continent['background_change_rank']
                
                try:
                    stat, p_val = mannwhitneyu(same_effects.dropna(), diff_effects.dropna(), alternative='two-sided')
                    print(f"Mann-Whitney U test: U = {stat:.1f}, p = {p_val:.3f}")
                    
                    if p_val < 0.05:
                        stronger = "same country" if abs(same_bg_effect) > abs(diff_bg_effect) else "different country"
                        print(f"**Significant difference: {stronger} participants show stronger effects**")
                except Exception as e:
                    print(f"Unable to compute test: {e}")
    
    def _pairwise_comparisons(self, trait_data, conditions):
        """Perform pairwise comparisons between conditions"""
        from itertools import combinations
        
        print("Pairwise comparisons:")
        condition_names = ['Original', 'Background', 'Text', 'Font']
        
        for i, (cond1, cond2) in enumerate(combinations(range(len(conditions)), 2)):
            ranks1 = trait_data[conditions[cond1]].dropna()
            ranks2 = trait_data[conditions[cond2]].dropna()
            
            if len(ranks1) >= 5 and len(ranks2) >= 5:
                try:
                    stat, p_val = wilcoxon(ranks1, ranks2, alternative='two-sided')
                    effect = ranks1.mean() - ranks2.mean()
                    print(f"  {condition_names[cond1]} vs {condition_names[cond2]}: p = {p_val:.3f}, effect = {effect:.3f}")
                except:
                    pass
    
    def create_comprehensive_visualizations(self):
        """
        Create comprehensive visualizations
        """
        key_traits = ['trustworthy', 'likable', 'competence']
        ranking_df = self.extract_phase2_rankings_by_politician(key_traits)
        
        if ranking_df.empty:
            print("No data for visualizations")
            return
        
        # Create a large figure with multiple subplots
        fig = plt.figure(figsize=(20, 15))
        gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)
        
        # 1. Overall rankings by condition
        ax1 = fig.add_subplot(gs[0, 0])
        conditions = ['original_rank', 'background_change_rank', 'text_change_rank', 'font_change_rank']
        condition_labels = ['Original', 'Background\nChange', 'Text\nChange', 'Font\nChange']
        
        avg_ranks = [ranking_df[cond].mean() for cond in conditions]
        bars = ax1.bar(condition_labels, avg_ranks, color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
        ax1.set_ylabel('Average Rank (lower = better)')
        ax1.set_title('Overall Rankings by Condition')
        ax1.set_ylim(0, 4)
        
        # 2 & 3. By politician comparison
        politicians = ranking_df['politician_name'].unique()
        politicians = [p for p in politicians if not pd.isna(p)][:2]  # Show top 2
        
        for i, politician in enumerate(politicians):
            ax = fig.add_subplot(gs[0, i+1])
            pol_data = ranking_df[ranking_df['politician_name'] == politician]
            
            pol_ranks = [pol_data[cond].mean() for cond in conditions]
            ax.bar(condition_labels, pol_ranks, alpha=0.7)
            ax.set_title(f'{politician}\nRankings')
            ax.set_ylabel('Average Rank')
            ax.set_ylim(0, 4)
        
        # 4. Geographical effects
        ax4 = fig.add_subplot(gs[1, 0])
        same_country_data = ranking_df[ranking_df['same_country'] == True]
        diff_country_data = ranking_df[ranking_df['same_country'] == False]
        
        if len(same_country_data) > 0 and len(diff_country_data) > 0:
            same_ranks = [same_country_data[cond].mean() for cond in conditions]
            diff_ranks = [diff_country_data[cond].mean() for cond in conditions]
            
            x = np.arange(len(condition_labels))
            width = 0.35
            
            ax4.bar(x - width/2, same_ranks, width, label='Same Country', alpha=0.8)
            ax4.bar(x + width/2, diff_ranks, width, label='Different Country', alpha=0.8)
            ax4.set_xlabel('Condition')
            ax4.set_ylabel('Average Rank')
            ax4.set_title('Same vs Different Country Effects')
            ax4.set_xticks(x)
            ax4.set_xticklabels(condition_labels)
            ax4.legend()
        
        # 5. Trait comparison
        ax5 = fig.add_subplot(gs[1, 1])
        trait_effects = {}
        for trait in key_traits:
            trait_data = ranking_df[ranking_df['trait'] == trait]
            if len(trait_data) > 0:
                bg_effect = trait_data['original_rank'].mean() - trait_data['background_change_rank'].mean()
                trait_effects[trait] = bg_effect
        
        if trait_effects:
            traits = list(trait_effects.keys())
            effects = list(trait_effects.values())
            ax5.bar(traits, effects, color='lightblue')
            ax5.set_ylabel('Background Effect Size')
            ax5.set_title('Background Color Effects by Trait')
            ax5.axhline(y=0, color='black', linestyle='-', alpha=0.3)
        
        # 6. Sample distribution
        ax6 = fig.add_subplot(gs[1, 2])
        continent_counts = ranking_df['participant_continent'].value_counts()
        if len(continent_counts) > 0:
            ax6.pie(continent_counts.values, labels=continent_counts.index, autopct='%1.1f%%')
            ax6.set_title('Sample Distribution by Continent')
        
        plt.suptitle('Comprehensive Political Perception Analysis', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.show()
    
    def run_recommended_analysis_pipeline(self):
        """
        Run the recommended analysis pipeline
        """
        print("COMPREHENSIVE POLITICAL PERCEPTION ANALYSIS")
        print("="*80)
        
        # Step 1: Pooled analysis for main effects
        print("\nSTEP 1: MAIN EFFECTS ANALYSIS")
        self.analyze_by_politician_strategy('pooled')
        
        # # Step 2: Individual politician analysis
        # print(f"\n{'='*80}")
        # print("STEP 2: INDIVIDUAL POLITICIAN EFFECTS")
        # self.analyze_by_politician_strategy('individual')
        
        # Step 3: Comparative analysis
        print(f"\n{'='*80}")
        print("STEP 3: COMPARATIVE ANALYSIS")
        self.analyze_by_politician_strategy('comparative')
        
        # Step 4: Enhanced geographical analysis
        # self.analyze_geographical_effects()
        
        # # Step 5: Visualizations
        print(f"\n{'='*80}")
        # print("CREATING COMPREHENSIVE VISUALIZATIONS")
        # self.create_comprehensive_visualizations()
        


In [14]:

phase2_data = [
    'phase2_data/luxon_phase2_data.csv',
    'phase2_data/biden_phase2_data.csv',
    'phase2_data/prayut_phase2_data.csv',
    'phase2_data/trump_phase2_data.csv',
    'phase2_data/friedrich_phase2_data.csv',
    'phase2_data/sunak_phase2_data.csv',
    'phase2_data/boko_phase2_data.csv',
    'phase2_data/morrison_phase2_data.csv',
    'phase2_data/olaf_phase2_data.csv',
    'phase2_data/singh_phase2_data.csv',
    'phase2_data/paetongtarn_phase2_data.csv',
    'phase2_data/bolsonaro_phase2_data.csv',
    'phase2_data/trudeau_phase2_data.csv',
    'phase2_data/starmer_phase2_data.csv',
    'phase2_data/albanese_phase2_data.csv',
    'phase2_data/modi_phase2_data.csv',
    'phase2_data/lee_phase2_data.csv',
    'phase2_data/ruto_phase2_data.csv',
    'phase2_data/yoon_phase2_data.csv',
    'phase2_data/kenyatta_phase2_data.csv',
    'phase2_data/jacinda_phase2_data.csv',
    'phase2_data/masisi_phase2_data.csv',
    'phase2_data/lula_phase2_data.csv',
    'phase2_data/harper_phase2_data.csv',
]


analyzer = ComprehensivePoliticalAnalyzer(phase2_data)

# Run the recommended analysis pipeline
analyzer.run_recommended_analysis_pipeline()

COMPREHENSIVE POLITICAL PERCEPTION ANALYSIS

STEP 1: MAIN EFFECTS ANALYSIS
Number of unique participants: 124
ANALYSIS STRATEGY: POOLED
POOLED ANALYSIS - All Politicians Combined
--------------------------------------------------

COMPETENCE Analysis:
Friedman test: χ² = 226.798, p = 0.000
Significant overall differences detected
Pairwise comparisons:
  Original vs Background: p = 0.000, effect = -0.835
  Original vs Text: p = 0.000, effect = -1.221
  Original vs Font: p = 0.000, effect = -1.607
  Background vs Text: p = 0.000, effect = -0.386
  Background vs Font: p = 0.000, effect = -0.772
  Text vs Font: p = 0.000, effect = -0.386

Descriptive Statistics:
Original: M = 1.58, SD = 0.86, n = 267
Background Change: M = 2.42, SD = 1.04, n = 267
Text Change: M = 2.81, SD = 0.92, n = 267
Font Change: M = 3.19, SD = 0.96, n = 267

LIKABLE Analysis:
Friedman test: χ² = 98.110, p = 0.000
Significant overall differences detected
Pairwise comparisons:
  Original vs Background: p = 0.000, effec